In [1]:
# Import necessary libraries for data handling and model creation
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K

In [2]:
df_train = pd.read_csv('train_final.csv', index_col=False)

# Extract labels from the last column 
labels = df_train[['784']]  

df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
2,0,0,0,0,0,0,0,0,0,14,...,0,0,0,0,0,0,0,0,0,10
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10


In [3]:
# Drop the label column from the DataFrame so we only have feature data
df_train.drop(df_train.columns[[784]], axis=1, inplace=True)

In [4]:
# Set a random seed for consistency in results
np.random.seed(1212)

In [5]:
labels = np.array(labels)
categorical_data = to_categorical(labels, num_classes=139)


In [6]:
# Prepare a list to hold reshaped images
l = []

# Loop through each row in df_train and reshape it to 28x28 images with 1 channel (grayscale)
for i in range(df_train.shape[0]):
    l.append(np.array(df_train[i:i+1]).reshape(28, 28, 1))

# Print the number of samples (images)
print("Number of samples (images):", len(l))


Number of samples (images): 188824


In [7]:
# Split the data into training and testing sets with 80% for training and 20% for testing
train_X, test_X, train_y, test_y = train_test_split(np.array(l), categorical_data, test_size=0.20, random_state=42)

# Print the lengths of the training and test datasets
print("Number of training samples:", len(train_X))
print("Number of training labels:", len(train_y))
print("Number of test samples:", len(test_X))

Number of training samples: 151059
Number of training labels: 151059
Number of test samples: 37765


In [8]:
np.random.seed(7)

In [9]:
model = keras.Sequential([
    layers.Conv2D(30, (5, 5), input_shape=(28, 28, 1), activation='relu'), 
    layers.MaxPool2D(pool_size=2), 
    layers.Conv2D(15, (3, 3), activation='relu'), 
    layers.MaxPool2D(pool_size=2), 
    layers.Dropout(0.2), 
    layers.Flatten(), 
    layers.Dense(128, activation='relu'), 
    layers.Dense(50, activation='relu'), 
    layers.Dense(139, activation='softmax'),  # Change to 139 units to match your labels
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#from tensorflow.keras.models import model_from_json

model.fit(train_X, train_y, validation_split=0.25, epochs=10, batch_size=200, shuffle=True, verbose=1)

c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
567/567 ━━━━━━━━━━━━━━━━━━━━ 34s 50ms/step - accuracy: 0.7176 - loss: 1.5282 - val_accuracy: 0.9532 - val_loss: 0.1645
Epoch 2/10
567/567 ━━━━━━━━━━━━━━━━━━━━ 29s 50ms/step - accuracy: 0.9448 - loss: 0.1884 - val_accuracy: 0.9684 - val_loss: 0.1085
Epoch 3/10
567/567 ━━━━━━━━━━━━━━━━━━━━ 39s 47ms/step - accuracy: 0.9594 - loss: 0.1352 - val_accuracy: 0.9738 - val_loss: 0.0898
Epoch 4/10
567/567 ━━━━━━━━━━━━━━━━━━━━ 45s 53ms/step - accuracy: 0.9664 - loss: 0.1093 - val_accuracy: 0.9761 - val_loss: 0.0793
Epoch 5/10
567/567 ━━━━━━━━━━━━━━━━━━━━ 28s 49ms/step - accuracy: 0.9717 - loss: 0.0950 - val_accuracy: 0.9780 - val_loss: 0.0733
Epoch 6/10
567/567 ━━━━━━━━━━━━━━━━━━━━ 27s 48ms/step - accuracy: 0.9741 - loss: 0.0843 - val_accuracy: 0.9802 - val_loss: 0.0643
Epoch 7/10
567/567 ━━━━━━━━━━━━━━━━━━━━ 28s 49ms/step - accuracy: 0.9755 - loss: 0.0788 - val_accuracy: 0.9819 - val_loss: 0.0590
Epoch 8/10
567/567 ━━━━━━━━━━━━━━━━━━━━ 45s 80ms/step - accuracy: 0.9768 - loss: 0.0739 - 

In [10]:
model.evaluate(test_X, test_y)

1181/1181 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9842 - loss: 0.0539


[0.05553325638175011, 0.9836621284484863]

In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 24, 24, 30)     │           780 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 12, 12, 30)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 10, 10, 15)     │         4,065 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 15)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 5, 5, 15)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 375)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        48,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 50)             │         6,450 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 139)            │         7,089 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 199,538 (779.45 KB)

 Trainable params: 66,512 (259.81 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 133,026 (519.64 KB)

In [12]:
from tensorflow.keras.models import model_from_json

In [16]:
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)

# Save the model weights with a filename ending in `.weights.h5`
model.save_weights('model.weights.h5')